# EmergencyZIP AI

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/patient-symptom-dataset/patient_symptom_data.csv


In [2]:
!pip uninstall -qqy jupyterlab  # Remove unused packages from Kaggle's base image that conflict
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.0 MB/s eta 0:00:00


In [3]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

In [4]:
from google.api_core import retry


is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [5]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

In [6]:
client = genai.Client(api_key=GOOGLE_API_KEY)

In [7]:
import typing_extensions as typing

class MedicalPatient(typing.TypedDict): 
    zipcode: int
    age: int
    gender: str
    symptoms: list[str]

def get_json_form(patient_symptom_entry): 
    entry_str = patient_symptom_entry.to_string()
    json_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=types.GenerateContentConfig(
            temperature=0.1, 
            response_mime_type="application/json", 
            response_schema=MedicalPatient
        ), 
        contents=entry_str
    )
    json_form = json_response.text
    return json_form

In [8]:
def get_general_model_config(temperature, top_p, max_output_tokens): 
    return types.GenerateContentConfig(
        temperature=temperature, 
        top_p=top_p, 
        max_output_tokens=max_output_tokens
    )

In [9]:
def few_shot_func(model_input, prompt, config): 
    few_shot_response = client.models.generate_content(
        model='gemini-2.0-flash', 
        config=config,
        contents=[prompt, model_input]
    )
    str_response = few_shot_response.text
    return str_response

In [10]:
def grounding_func(input_prompt): 
    config_with_search = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
    )
    response = client.models.generate_content(
        model='gemini-2.0-flash', 
        contents=input_prompt, 
        config=config_with_search,
    )
    rc = response.candidates[0]
    return rc.content.parts[0].text

In [11]:
patient_symptom_df = pd.read_csv('/kaggle/input/patient-symptom-dataset/patient_symptom_data.csv')
patient_symptom_df

,PatientID,ZipCode,Age,Gender,Symptoms
0,P001,70112,32,Female,Joint pain;Runny nose;Loss of taste;Sore throat
1,P002,15213,87,Female,Cough;Fever;Fatigue;Chest pain;Joint pain
2,P003,46201,21,Non-binary,Swelling;Blurred vision;Rash
3,P004,20001,46,Male,Chest tightness;Sneezing;Fever;Breathing diffi...
4,P005,96813,38,Non-binary,Vomiting;Headache;Runny nose;Chest pain;Breath...
...,...,...,...,...,...
95,P096,92101,71,Non-binary,Low-grade fever;Joint pain;Headache
96,P097,20001,80,Female,Vomiting;Abdominal cramps;Chest tightness;Abdo...
97,P098,85001,77,Female,Loss of smell;Abdominal pain
98,P099,84101,90,Non-binary,Fatigue;Blurred vision


In [12]:
patient_index = random.randint(0, 99)
patient_json_form = get_json_form(patient_symptom_df.iloc[patient_index])
print(patient_json_form)

{
"zipcode": 68102,
"age": 27,
"gender": "Non-binary",
"symptoms": ["Rash", "Loss of taste", "Joint pain"]
}


In [13]:
model_config = get_general_model_config(0.1, 1, 500)

few_shot_prompt = """Parse JSON into a string request and only return output: 

EXAMPLE: 
{
"zipcode": 78664, 
"age": 30, 
"gender": "Female", 
"symptoms": ["Cough", "Fever", "Sore throat"]
}
"30-year old female patient from zipcode 78664 currently experiencing symptoms of cough, fever, and sore throat."

EXAMPLE: 
{
"zipcode": 77055, 
"age": 25, 
"gender": "Non-binary", 
"symptoms": ["Joint pain", "Loss of smell"]
}
"25-year old non-binary patient from zipcode 77055 currently experiencing symptoms of joint pain and loss of smell."

EXAMPLE: 
{
"zipcode": 78681, 
"age": 15, 
"gender" "Male", 
"symptoms": ["Fatigue", "Loss of Vision", "Headache", "Joint pain"]
}
"15-year old male patient from zipcode 78681 currently experiencing symptoms of fatigue, loss of vision, headache, and joint pain."
"""

patient_medical_request = few_shot_func(patient_json_form, few_shot_prompt, model_config)
print(patient_medical_request)

"27-year old non-binary patient from zipcode 68102 currently experiencing symptoms of rash, loss of taste, and joint pain."



In [14]:
diagnosis_prompt = patient_medical_request + "What disease are they most likely to experience now? Only return the singular most likely disease, please."
diagnosis = grounding_func(diagnosis_prompt)
Markdown(diagnosis)

Based on the symptoms of rash, loss of taste, and joint pain, a few possibilities should be considered. However, given the available information, the most likely singular disease is:

*   **COVID-19:** Loss of taste and rash are symptoms of COVID-19.
*   **Adult Still's Disease:** This is a rare inflammatory condition that presents with fever, rash, and joint pain.
*   **Fifth Disease:** This viral illness can cause a rash and joint pain, especially in adults.
*   **Long COVID:** Joint pain, rash, and loss of taste may be symptoms of Long COVID.

Without more information, it is difficult to determine the most likely diagnosis. Consulting a healthcare professional is essential for accurate diagnosis and treatment.


In [15]:
model_config_two = get_general_model_config(0, 1, 50)

few_shot_prompt_two = """Parse JSON to return only the Zipcode: 

EXAMPLE: 
{
"zipcode": 78664, 
"age": 30, 
"gender": "Female", 
"symptoms": ["Cough", "Fever", "Sore throat"]
}
78664

EXAMPLE: 
{
"zipcode": 77055, 
"age": 25, 
"gender": "Non-binary", 
"symptoms": ["Joint pain", "Loss of smell"]
}
77055

EXAMPLE: 
{
"zipcode": 78681, 
"age": 15, 
"gender" "Male", 
"symptoms": ["Fatigue", "Loss of Vision", "Headache", "Joint pain"]
}
78681
"""

patient_zipcode = few_shot_func(patient_json_form, few_shot_prompt_two, model_config_two)
print(patient_zipcode)

68102



In [16]:
recommendations_prompt = diagnosis + 'Can you get me the nearest best-rated medical facilities with their addresses that can treat the disease for zip code ' + str(patient_zipcode) + '?'
recommendations = grounding_func(recommendations_prompt)
Markdown(recommendations)

I understand you're looking for the best-rated medical facilities near the 68102 zip code that can address the symptoms you described: rash, loss of taste, and joint pain. Based on your listed possible conditions (COVID-19, Adult Still's Disease, Fifth Disease, and Long COVID), here's a breakdown of facilities that may be helpful:

**Please note:** It's essential to confirm with any facility that they are currently accepting new patients and that they have expertise in treating your specific symptoms or suspected condition. Contacting them directly is always a good first step.

**General Medical Facilities in/near 68102**

*   **Nebraska Medicine:** Nebraska Medicine has multiple primary care locations, including one at 1319 Leavenworth St, Omaha, NE 68102. They offer a team-based approach to primary care.
*   **Methodist Urgent Care Downtown Omaha:** Located at 312 S. 15th St. Omaha, NE 68102, this urgent care center can treat a range of minor medical issues, including rashes and cold/flu symptoms. It's an option for immediate care without an appointment.
*   **CHI Health Center Omaha:** While primarily a convention center, it's worth noting that many hospitals and medical facilities in the area are part of the CHI Health network. Consider exploring CHI Health providers in the area. The address is 455 N 10th St Omaha, NE 68102.
*   **Magis Clinic:** This free clinic, run by Creighton University medical students, is located at Siena Francis House, 1401 North 18th Street, Omaha, NE 68102. It provides basic acute primary care consultations for the homeless and uninsured on Saturdays.

**COVID-19 and Long COVID Care**

*   **Nebraska Medicine:** As an academic medical center, Nebraska Medicine is likely equipped to diagnose and manage COVID-19 and Long COVID. Contact their primary care or internal medicine departments.
*   **Methodist Health System:** Methodist has multiple locations and may offer services for COVID-19 and related complications.
*   **Post-COVID Clinics (out of state):** While not in your immediate area, it's useful to know that specialized Post-COVID clinics exist. For example, MetroHealth in Cleveland, OH, offers virtual appointments for those experiencing long-term effects of COVID-19. Also, The University of Utah Health has a COVID-19 clinic.

**Adult Still's Disease**

*   **Rheumatology Specialists:** Adult Still's Disease is a rare inflammatory condition, so seeing a rheumatologist is crucial. Look for rheumatology specialists at Nebraska Medicine, CHI Health, or Methodist Health System.
*   **Cleveland Clinic (out of state):** The Cleveland Clinic has a dedicated team for Adult-Onset Still's Disease. While it would require travel, their expertise may be valuable, especially for diagnosis and complex cases.

**Fifth Disease**

*   **Primary Care Physician/Pediatrician:** Fifth disease is usually a mild viral illness. Your primary care physician can typically diagnose and manage it. For children, a pediatrician is the appropriate choice. Health Care for Children in Kansas City, MO, also provide treatment for Fifth Disease.

**Important Considerations:**

*   **COVID-19 Testing:** If you suspect COVID-19, prioritize getting tested and following public health guidelines.
*   **Referrals:** Some specialists or Post-COVID clinics may require a referral from your primary care physician.
*   **Insurance:** Verify that the facility accepts your insurance plan.

I strongly recommend consulting your primary care physician or using your insurance provider's website to find in-network medical facilities and specialists in the 68102 area. They can guide you toward the most appropriate care based on your specific symptoms and medical history.
